In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

data=pd.read_csv('/kaggle/input/merged/merged_data.csv')
data.set_index('Date', inplace=True)
train_data, test_data = train_test_split(data, test_size=492, random_state=42)
print('ok')

ok


In [ ]:
train_data

In [ ]:
test_data 

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


**GPT2 Models**

In [3]:

from transformers import GPT2Tokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def prepare_data(data, tokenizer, max_seq_length):
    tokenized = tokenizer(
        list(data['clean text']), padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt"
        )
    labels = torch.tensor(
        data['increase_classes'].values,
        dtype=torch.int64
        )
    return TensorDataset(tokenized['input_ids'], tokenized['attention_mask'], labels)


max_seq_length = 128

# Подготовка данных
train_dataset = prepare_data(train_data, tokenizer, max_seq_length)
test_dataset = prepare_data(test_data, tokenizer, max_seq_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
print('ok')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

ok


In [4]:
from transformers import GPT2Model
import torch.nn as nn


# Определение модели
class GPT2Small(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt = GPT2Model.from_pretrained('gpt2')
        self.classifier = nn.Sequential(
            nn.Linear(self.gpt.config.n_embd,64),
            nn.ReLU(),
            nn.Linear(64,3),
            nn.Softmax(dim=1)
        )

    def forward(self, input_ids, attention_mask):
        gpt_output = self.gpt(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = gpt_output.last_hidden_state[:, 0]
        predictions = self.classifier(pooled_output)
        return predictions
model=GPT2Small().to(device)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [12]:
from transformers import GPT2Model
import torch.nn as nn


# Определение модели
class GPT2Medium(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt = GPT2Model.from_pretrained('gpt2')
        self.fc1 = nn.Linear(self.gpt.config.n_embd,256)
        self.act1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(256,128)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(128,3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        gpt_output = self.gpt(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = gpt_output.last_hidden_state[:, 0]
        x = self.fc1(pooled_output)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        predictions = self.softmax(x)
        return predictions
model=GPT2Medium().to(device)

print('ok')

ok


In [20]:
from transformers import GPT2Model
import torch.nn as nn


# Определение модели
class GPT2Big(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt = GPT2Model.from_pretrained('gpt2')
        self.fc1 = nn.Linear(self.gpt.config.n_embd,512)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512,256)
        self.act2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(256,128)
        self.act3 = nn.ReLU()
        self.fc4 = nn.Linear(128,3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        gpt_output = self.gpt(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = gpt_output.last_hidden_state[:, 0]
        x = self.fc1(pooled_output)
        x = self.act1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        x = self.act3(x)
        x = self.fc4(x)
        predictions = self.softmax(x)
        return predictions
model=GPT2Big().to(device)

print('ok')

ok


**RoBERTa Models**

In [ ]:

from transformers import RobertaTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def prepare_data(data, tokenizer, max_seq_length):
    tokenized = tokenizer(
        list(data['clean text']), padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt"
        )
    labels = torch.tensor(
        data['increase_classes'].values,
        dtype=torch.int64
        )
    return TensorDataset(tokenized['input_ids'], tokenized['attention_mask'], labels)


max_seq_length = 128

# Подготовка данных
train_dataset = prepare_data(train_data, tokenizer, max_seq_length)
test_dataset = prepare_data(test_data, tokenizer, max_seq_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
print('ok')

In [ ]:
from transformers import RobertaModel
import torch.nn as nn


# Определение модели
class RobertaSmall(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size,64),
            nn.Linear(64, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, input_ids, attention_mask):
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        predictions = self.classifier(pooled_output)
        return predictions
model=RobertaSmall().to(device)

print('ok')

In [ ]:
from transformers import RobertaModel
import torch.nn as nn


# Определение модели
class RobertaBig(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.fc1 = nn.Linear(self.roberta.config.hidden_size,512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512,256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256,90)
        self.fc4 = nn.Linear(90,3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        x = self.fc1(pooled_output)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        predictions = self.softmax(x)
        return predictions
model=RobertaBig().to(device)

print('ok')

**BERT Models**

In [ ]:

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def prepare_data(data, tokenizer, max_seq_length):
    tokenized = tokenizer(
        list(data['clean text']), padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt"
        )
    labels = torch.tensor(
        data['increase_classes'].values,
        dtype=torch.int64
        )
    return TensorDataset(tokenized['input_ids'], tokenized['attention_mask'], labels)


max_seq_length = 128

# Подготовка данных
train_dataset = prepare_data(train_data, tokenizer, max_seq_length)
test_dataset = prepare_data(test_data, tokenizer, max_seq_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
print('ok')

In [ ]:
from transformers import BertModel
import torch.nn as nn


# Определение модели
class BERTSmall(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size,64),
            nn.Linear(64, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        predictions = self.classifier(pooled_output)
        return predictions
model=BERTSmall().to(device)

print('ok')

**Organizing for traininig**

In [21]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, X in enumerate(dataloader):
        input_ids, attention_mask, labels = [x.to(device) for x in X]
        pred=model(input_ids,attention_mask)
        loss=loss_fn(pred,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(input_ids)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader,model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            pred = model(input_ids,attention_mask)
            test_loss+=loss_fn(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, correct

print('ok')

ok


In [22]:
input_ids, attetion_mask, labels = next(iter(train_loader))
input_ids=input_ids.to(device)
attetion_mask=attetion_mask.to(device)
labels=labels.to(device)


In [23]:
pred=model(input_ids, attetion_mask)
print(pred)
res=pred.argmax(1)
print(res)

tensor([[0.3861, 0.3697, 0.2441],
        [0.3357, 0.3251, 0.3392],
        [0.2991, 0.3815, 0.3194],
        [0.3526, 0.3576, 0.2899],
        [0.3285, 0.3578, 0.3137],
        [0.3080, 0.4166, 0.2754],
        [0.3010, 0.4010, 0.2980],
        [0.2787, 0.3928, 0.3285],
        [0.3324, 0.3994, 0.2682],
        [0.2737, 0.3909, 0.3354],
        [0.4036, 0.3178, 0.2786],
        [0.3397, 0.3711, 0.2892],
        [0.2707, 0.3524, 0.3769],
        [0.3339, 0.3740, 0.2921],
        [0.3556, 0.3512, 0.2933],
        [0.2884, 0.4418, 0.2699],
        [0.3007, 0.3725, 0.3268],
        [0.3002, 0.4047, 0.2952],
        [0.3364, 0.3219, 0.3417],
        [0.3569, 0.3583, 0.2849],
        [0.3387, 0.3156, 0.3457],
        [0.3212, 0.3598, 0.3190],
        [0.3627, 0.3383, 0.2989],
        [0.3662, 0.3311, 0.3027],
        [0.3944, 0.3674, 0.2382],
        [0.2394, 0.4590, 0.3016],
        [0.3072, 0.3507, 0.3421],
        [0.3346, 0.3599, 0.3054],
        [0.3344, 0.3854, 0.2802],
        [0.412

In [24]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lambda1 = lambda epoch: 0.90 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
loss_fn = nn.CrossEntropyLoss()
print('ok')


ok


In [25]:
test_l = []
accuracy = []

In [26]:

epochs = 25
for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      model.train(True)
      train_loop(train_loader, model, loss_fn, optimizer)
      scheduler.step()
      model.train(False)
      t,a =test_loop(test_loader, model, loss_fn)
      test_l.append(t)
      accuracy.append(a)
print("Done!")


Epoch 1
-------------------------------
loss: 1.136303  [    0/ 1472]
loss: 1.091820  [  320/ 1472]
loss: 1.009764  [  640/ 1472]
loss: 0.963723  [  960/ 1472]
loss: 1.018344  [ 1280/ 1472]
Test Error: 
 Accuracy: 50.8%, Avg loss: 0.964905 

Epoch 2
-------------------------------
loss: 0.952297  [    0/ 1472]
loss: 0.992125  [  320/ 1472]
loss: 0.968071  [  640/ 1472]
loss: 0.974693  [  960/ 1472]
loss: 0.966627  [ 1280/ 1472]
Test Error: 
 Accuracy: 50.8%, Avg loss: 0.963104 

Epoch 3
-------------------------------
loss: 0.955575  [    0/ 1472]
loss: 0.965947  [  320/ 1472]
loss: 0.954455  [  640/ 1472]
loss: 0.954709  [  960/ 1472]
loss: 0.976778  [ 1280/ 1472]
Test Error: 
 Accuracy: 48.4%, Avg loss: 0.964676 

Epoch 4
-------------------------------
loss: 0.974004  [    0/ 1472]
loss: 0.981209  [  320/ 1472]
loss: 0.953664  [  640/ 1472]
loss: 0.970518  [  960/ 1472]
loss: 0.949536  [ 1280/ 1472]
Test Error: 
 Accuracy: 50.8%, Avg loss: 0.963802 

Epoch 5
------------------------

In [27]:
import numpy as np
test_loss_np = np.array(test_l)
accuracy_np = np.array(accuracy)
np.save('gpt_big_loss', test_loss_np)
np.save('gpt_big_accuracy', accuracy_np)

In [19]:
del model


In [ ]:
print(torch.cuda.memory_allocated(device))

In [ ]:
torch.save(model.state_dict(), "model.pt")